**Inputs**
* NEX-GDDP data
* Hazard definition(s)

**Outputs**
* ImageCollection(s) containing hazard magnitudes for each year-model pair
* Stored in users/tedwongwri/dataportal/magnitudes/HAZARDNAME
* **You must manually create the empty ImageCollection** in the dataportal/magnitudes directory before trying to export results

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWgNY2iP_hneYL3QjkCRzGd9a-jcDNQDXrL6mfjVI2Zv5bz5Bk2FRFM

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [12]:
# Define hazards here

def dryspells(yeardata):
    dry_days = yeardata.map(lambda d: d.eq(0))
    return process_runs(dry_days, 5, 'count')

def ECE(yeardata):
    return

def maxdryspell(yeardata):
    # Climdex CDD: Let RR_ij be the daily precipitation amount on day i in period j. Count the largest number of consecutive days where RR_ij < 1mm
    dry_days = yeardata.map(lambda d: d.multiply(86400).lt(1))
    return process_runs(dry_days, 1, 'max')

hazards = [
            {'definition': maxdryspell,
             'variable': 'pr'
            },
]

In [20]:
def process_runs(imgs, runLength, resultType): 
# from Logan Byers
    def doOne(img, data):
   #data = ee.Image(data);

        dataDict = ee.Dictionary(data)

        previousThresholdImage = ee.Image(dataDict.get('previousThresholdImage'))
        currentStreakImage = ee.Image(dataDict.get('currentStreakImage')).uint16()
        streakCountImage = ee.Image(dataDict.get('streakCountImage')).uint16()
        longestStreakImage = ee.Image(dataDict.get('longestStreakImage')).uint16()
        streakAccumulation = ee.Image(dataDict.get('streakAccumulation')).uint16()
        numRemaining = ee.Number(dataDict.get('numRemaining'))

        # WHERE yesterday AND today : 1, else 0
        #continueStreakImage = previousThresholdImage.and(ee.Image(img))
        continueStreakImage = previousThresholdImage.multiply(ee.Image(img)).multiply(ee.Image(numRemaining).gt(1).multiply(1))

        # WHERE NOT on streak :  yesterday streak length, else 0
        #streakEndedImage = currentStreakImage.multiply(currentStreakImage.and(continueStreakImage.not()))
        streakEndedImage = currentStreakImage.multiply(currentStreakImage.multiply(continueStreakImage.multiply(-1).add(1)))

        # WHERE NOT on streak AND yesterday streak length > length threshold : 1, else 0
        endedStreakExceedsLengthImage = currentStreakImage.multiply(streakEndedImage).gte(runLength)

        # update the state
        accumulator = ee.Dictionary.fromLists([
            'previousThresholdImage',
            'currentStreakImage',
            'streakCountImage',
            'longestStreakImage',
            'streakAccumulation',
            'numRemaining'
          ], [
            # previousThresholdImage --> today's image
            ee.Image(img),
            # currentStreakImage --> today's image PLUS yesterday's streak (where continuing)
            currentStreakImage
              .multiply(continueStreakImage).add(ee.Image(img)),
            # streakCountImage --> PLUS 1 where long streak ended today
            streakCountImage.add(endedStreakExceedsLengthImage),
            # longestStreakImage --> larger of prev and current value
            longestStreakImage.max(currentStreakImage.multiply(continueStreakImage).add(ee.Image(img))),
            # streakAccumulation --> yesterday's accum plus current 1/0, if in streak
            streakAccumulation.add(ee.Image(img).multiply(continueStreakImage)),
            # numRemaining --> minus 1
            numRemaining.add(-1)
          ]
        )

        return accumulator
  
    resultImageName = {
        'count': 'streakCountImage',
        'max': 'longestStreakImage',
        'accum': 'streakAccumulation'
    }
  
    streakData = imgs.iterate(
    # iterate over each image in the ImageCollection
    #   accumulate a stateful Dictionary of images
    
        doOne, ee.Dictionary.fromLists([
          'previousThresholdImage', 
          'currentStreakImage',
          'streakCountImage',
          'longestStreakImage',
          'streakAccumulation',
          'numRemaining'
        ], [
          ee.Image.constant(1),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          imgs.size().add(-1)
        ]
        )
    
    )
    return ee.Image(ee.Dictionary(streakData).get(resultImageName[resultType]))

In [5]:
NEX = ee.ImageCollection("NASA/NEX-GDDP")
PR_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/pr_99')
TASMAX_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmax_99'),
TASMIN_01 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmin_01'),
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
START_YEAR = 2010
END_YEAR = 2099

In [ ]:
for hazard in hazards:
    for year in range(START_YEAR, END_YEAR + 1):
        for model in MODELS:
            data = NEX.filterMetadata('model', 'equals', model).filterDate(str(year) + '-01-01', str(year) + '-12-31').filterMetadata('scenario', 'equals', ['historical', 'rcp85'][(year > 2005) * 1]).select(hazard['variable'])
            img = hazard['definition'](data)
            task = ee.batch.Export.image.toAsset(**{
              'image': img.rename(hazard['definition'].__name__).set('model', model).set('year', year),
              'description': '{0}_{1}_{2}'.format(hazard['definition'].__name__, str(year), model),
              'assetId': 'users/tedwongwri/dataportal/magnitudes/{0}/{1}_{2}'.format(hazard['definition'].__name__, str(year), model),
              'region': WHOLE_GLOBE,
              'crs': 'EPSG:4326',
              'dimensions': '1440x720'
            })
            task.start()